# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-25 13:25:55] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=30618, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=39216204, constrained_json_whitespace_pattern=

[2025-07-25 13:25:55] Inferred chat template from model path: llama-2


[2025-07-25 13:26:11] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-25 13:26:11] Init torch distributed begin.
[2025-07-25 13:26:11] Init torch distributed ends. mem usage=0.00 GB


[2025-07-25 13:26:12] Load weight begin. avail mem=62.08 GB


[2025-07-25 13:26:12] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-07-25 13:26:16] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.13s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.14s/it]

[2025-07-25 13:26:17] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.72 GB, mem usage=0.93 GB.
[2025-07-25 13:26:17] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-25 13:26:17] Memory pool end. avail mem=37.40 GB


[2025-07-25 13:26:18] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=37.63 GB
[2025-07-25 13:26:18] INFO:     Started server process [1301896]
[2025-07-25 13:26:18] INFO:     Waiting for application startup.
[2025-07-25 13:26:18] INFO:     Application startup complete.
[2025-07-25 13:26:18] INFO:     Uvicorn running on http://127.0.0.1:30618 (Press CTRL+C to quit)


[2025-07-25 13:26:18] INFO:     127.0.0.1:59328 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-25 13:26:19] INFO:     127.0.0.1:59332 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-25 13:26:19] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-25 13:27:23] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-07-25 13:27:52] INFO:     127.0.0.1:59340 - "POST /generate HTTP/1.1" 200 OK
[2025-07-25 13:27:52] The server is fired up and ready to roll!


[2025-07-25 13:27:55] INFO:     127.0.0.1:59344 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-07-25 13:27:55] Child process unexpectedly failed with exitcode=9. pid=1302551


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-25 13:28:07] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=36719, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=515891660, constrained_json_whitespace_pattern=N

[2025-07-25 13:28:07] Inferred chat template from model path: llama-2


[2025-07-25 13:28:20] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-25 13:28:20] Init torch distributed begin.


[2025-07-25 13:28:20] Init torch distributed ends. mem usage=0.00 GB


[2025-07-25 13:28:21] Load weight begin. avail mem=61.31 GB


...

[2025-07-25 13:28:25] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.19s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.19s/it]

[2025-07-25 13:28:26] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.04 GB, mem usage=0.93 GB.
[2025-07-25 13:28:26] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-25 13:28:26] Memory pool end. avail mem=36.73 GB


[2025-07-25 13:28:26] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=36.91 GB
[2025-07-25 13:28:27] INFO:     Started server process [1309666]
[2025-07-25 13:28:27] INFO:     Waiting for application startup.
[2025-07-25 13:28:27] INFO:     Application startup complete.
[2025-07-25 13:28:27] INFO:     Uvicorn running on http://127.0.0.1:36719 (Press CTRL+C to quit)


[2025-07-25 13:28:27] INFO:     127.0.0.1:49102 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-25 13:28:28] INFO:     127.0.0.1:49106 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-25 13:28:28] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-25 13:28:31] INFO:     127.0.0.1:49120 - "POST /generate HTTP/1.1" 200 OK
[2025-07-25 13:28:31] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-25 13:28:32] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-25 13:28:33] INFO:     127.0.0.1:49122 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-07-25 13:28:33] Child process unexpectedly failed with exitcode=9. pid=1310096


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-25 13:28:48] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=33595, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=554451936, constrained_json_white

[2025-07-25 13:29:00] Casting torch.bfloat16 to torch.float16.


[2025-07-25 13:29:01] Casting torch.bfloat16 to torch.float16.
[2025-07-25 13:29:01] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-25 13:29:01] Init torch distributed begin.


[2025-07-25 13:29:04] Init torch distributed ends. mem usage=0.31 GB


[2025-07-25 13:29:05] Load weight begin. avail mem=50.32 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.02s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.02s/it]

[2025-07-25 13:29:21] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=41.42 GB, mem usage=1.70 GB.
[2025-07-25 13:29:21] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-07-25 13:29:21] Memory pool end. avail mem=41.34 GB


[2025-07-25 13:29:22] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=42.02 GB


[2025-07-25 13:29:23] INFO:     Started server process [1311446]
[2025-07-25 13:29:23] INFO:     Waiting for application startup.
[2025-07-25 13:29:23] INFO:     Application startup complete.
[2025-07-25 13:29:23] INFO:     Uvicorn running on http://127.0.0.1:33595 (Press CTRL+C to quit)


[2025-07-25 13:29:24] INFO:     127.0.0.1:35832 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-25 13:29:24] INFO:     127.0.0.1:35840 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-25 13:29:24] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-25 13:29:27] INFO:     127.0.0.1:35844 - "POST /generate HTTP/1.1" 200 OK
[2025-07-25 13:29:27] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-25 13:29:29] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-25 13:29:29] INFO:     127.0.0.1:57790 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-25 13:29:41] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=35172, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=671178449, constrained_json_whitespace_

[2025-07-25 13:29:54] Casting torch.bfloat16 to torch.float16.


[2025-07-25 13:29:54] Casting torch.bfloat16 to torch.float16.
[2025-07-25 13:29:54] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-25 13:29:54] Init torch distributed begin.


[2025-07-25 13:29:54] Init torch distributed ends. mem usage=0.00 GB


[2025-07-25 13:29:55] Load weight begin. avail mem=59.43 GB


...

[2025-07-25 13:30:11] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.56 GB


[2025-07-25 13:30:11] INFO:     Started server process [1314368]
[2025-07-25 13:30:11] INFO:     Waiting for application startup.
[2025-07-25 13:30:11] INFO:     Application startup complete.
[2025-07-25 13:30:11] INFO:     Uvicorn running on http://127.0.0.1:35172 (Press CTRL+C to quit)


[2025-07-25 13:30:12] INFO:     127.0.0.1:60986 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-25 13:30:12] INFO:     127.0.0.1:32768 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-25 13:30:12] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-25 13:30:15] INFO:     127.0.0.1:32776 - "POST /generate HTTP/1.1" 200 OK
[2025-07-25 13:30:15] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-25 13:30:17] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-25 13:30:18] INFO:     127.0.0.1:45210 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-25 13:30:18] Child process unexpectedly failed with exitcode=9. pid=1315121


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-25 13:30:29] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=31402, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=516844836, constrained_json_whitespace_pattern=None, watchdog_timeo

[2025-07-25 13:30:42] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-07-25 13:30:42] Init torch distributed begin.
[2025-07-25 13:30:43] Init torch distributed ends. mem usage=0.00 GB


[2025-07-25 13:30:43] Load weight begin. avail mem=78.50 GB


[2025-07-25 13:30:44] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.60it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  9.02it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  8.58it/s]

[2025-07-25 13:30:48] Load weight end. type=MiMoMTP, dtype=torch.bfloat16, avail mem=57.83 GB, mem usage=2.71 GB.
[2025-07-25 13:30:48] KV Cache is allocated. #tokens: 20480, K size: 1.41 GB, V size: 1.41 GB
[2025-07-25 13:30:48] Memory pool end. avail mem=54.88 GB


[2025-07-25 13:30:48] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-07-25 13:30:49] INFO:     Started server process [1315908]
[2025-07-25 13:30:49] INFO:     Waiting for application startup.
[2025-07-25 13:30:49] INFO:     Application startup complete.
[2025-07-25 13:30:49] INFO:     Uvicorn running on http://0.0.0.0:31402 (Press CTRL+C to quit)
[2025-07-25 13:30:49] INFO:     127.0.0.1:52534 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-25 13:30:50] INFO:     127.0.0.1:52550 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-25 13:30:50] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-25 13:30:51] INFO:     127.0.0.1:52558 - "POST /generate HTTP/1.1" 200 OK
[2025-07-25 13:30:51] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-25 13:30:54] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-07-25 13:30:54] Decode batch. #running-req: 1, #token: 90, token usage: 0.00, accept len: 1.74, cuda graph: False, gen throughput (token/s): 11.38, #queue-req: 0, 


[2025-07-25 13:30:55] Decode batch. #running-req: 1, #token: 157, token usage: 0.01, accept len: 1.68, cuda graph: False, gen throughput (token/s): 103.42, #queue-req: 0, 


[2025-07-25 13:30:56] Decode batch. #running-req: 1, #token: 224, token usage: 0.01, accept len: 1.68, cuda graph: False, gen throughput (token/s): 100.28, #queue-req: 0, 


[2025-07-25 13:30:56] Decode batch. #running-req: 1, #token: 288, token usage: 0.01, accept len: 1.60, cuda graph: False, gen throughput (token/s): 96.04, #queue-req: 0, 


[2025-07-25 13:30:57] Decode batch. #running-req: 1, #token: 358, token usage: 0.02, accept len: 1.75, cuda graph: False, gen throughput (token/s): 106.20, #queue-req: 0, 


[2025-07-25 13:30:58] Decode batch. #running-req: 1, #token: 429, token usage: 0.02, accept len: 1.77, cuda graph: False, gen throughput (token/s): 107.83, #queue-req: 0, 


[2025-07-25 13:30:58] INFO:     127.0.0.1:52568 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-07-25 13:30:58] Child process unexpectedly failed with exitcode=9. pid=1316193


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).